In [1]:
# all necessary requestcompletion imports 
import src.requestcompletion as rc
from src.requestcompletion.llm import MessageHistory, UserMessage, SystemMessage
from src.requestcompletion.visuals.agent_viewer import AgentViewer
from src.requestcompletion.nodes.library import from_function

<img src="./assets/Telus-Symbol.png" alt="Telus Symbol" width="200"/>

## RC Demo: Customer Service Ticketing Agent + Notion:

In this demo we will demonstrate how to use RequestCompletion to built AI Agents and combine them to perrform complex tasks using just a few lines of code.

### Step 1: Import and use Notion Agent from sample_agents

In [ ]:
from demo.sample_agents import NotionAgent

### Step 2: Invoke the agent 🚀

In [ ]:
USER_MESSAGE = UserMessage("Under the Telus Demo page, add a new page called 'RC is Awesome'. Inside that page write something awesome about RC.")

In [ ]:
message_history = MessageHistory([USER_MESSAGE])

with rc.Runner() as runner1:
    result = await runner1.run(NotionAgent, message_history)

we can also visualize the agent control flow

In [ ]:
viewer = AgentViewer(
        stamps=result.all_stamps,
        request_heap=result.request_heap,
        node_heap=result.node_heap
    )
viewer.display_graph()

### Now lets build our own Agent for Telus

We have some mock functions that we will give the Agent access to as tools.

In [ ]:
from demo.sample_tools.telus_tools import check_network_status, analyze_call_volume, create_service_ticket, get_service_tickets
from demo.sample_tools.notion_tools import NotionAgenticTool    # use the NotionAgent as a tool

In [ ]:
# create mock data (telus Database)
create_service_ticket("internet_outage", "Vancouver, BC")
create_service_ticket("billing", "Vancouver, BC")
create_service_ticket("device_support", "Langley, BC")
create_service_ticket("internet_outage", "Surrey, BC")
create_service_ticket("tv_service", "Vancouver, BC")

In [ ]:
USER_MESSAGE = UserMessage("Analyze the call volume and network status for the Vancouver area this week." \
               "Categorize the service tickets by issue type and create a summary page showing resolution times and identifying any recurring problems." \
               "Give me an analysis page and a summary page. The notion root page is 'Telus Demo'.")

In [ ]:
SYSTEM_TELUS = SystemMessage("You are a helpful Telus agent. You have access to the following tools: check_network_status," \
"analyze_call_volume, create_service_ticket, get_service_tickets, and NotionTool. Your goal is to resolve the user's request as efficiently as possible." \
"Today is April 25th 2025.")

In [ ]:
TelusTicketingAgent = rc.library.tool_call_llm(connected_nodes={from_function(check_network_status), 
                                                                from_function(analyze_call_volume),
                                                                from_function(create_service_ticket),
                                                                from_function(get_service_tickets),
                                                                NotionAgenticTool},
                                               pretty_name="Telus Ticket Analyzer",
                                               system_message= SYSTEM_TELUS,
                                               model=rc.llm.OpenAILLM("gpt-4o"),
                                               )

In [ ]:
with rc.Runner(executor_config=rc.ExecutorConfig(timeout=50)) as runner2:
    result = await runner2.run(TelusTicketingAgent, MessageHistory([UserMessage(USER_MESSAGE)]))

In [ ]:
viewer = AgentViewer(
        stamps=result.all_stamps,
        request_heap=result.request_heap,
        node_heap=result.node_heap
    )
viewer.display_graph()